In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
credits=pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv')
movies=pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv')

In [ ]:
print(movies.shape)

In [ ]:
movies.head(1)

In [ ]:
print(credits.shape)
credits.head()

In [ ]:
credits.isnull().sum()

In [ ]:
credits=credits[['movie_id','cast','crew']]

In [ ]:
credits.rename(columns={'movie_id':'id'},inplace=True)
credits.head()

In [ ]:
df=movies.merge(credits,on='id')

In [ ]:
df.sample(1)

In [ ]:
# General IMDB wala
# Content based system

In [ ]:
# weighted rating (WR) = [(v/(v+m)) × R] + [(m/(v+m)) × C]

# Where:

# R = average for the movie (mean) = (rating)

# v = number of votes for the movie = (votes)

# m = minimum votes required to be listed in the Top Rated list (currently 25,000)

# C = the mean vote across the whole report

In [ ]:
m=25000
C=df['vote_average'].mean()

In [ ]:
def weighted_rating(row,m=m,C=C):
    
    R=row['vote_average']
    v=row['vote_count']
    
    return ((v/(v+m))*R) + ((m/(v+m))*C)

In [ ]:
df['wr']=df.apply(weighted_rating,axis=1)

In [ ]:
df[['title','wr']].sort_values('wr',ascending=False)

In [ ]:
# Important cols for content based filtering

# 1. Genre
# 2. keywords
# 3. overview
# 4. cast
# 5. crew

In [ ]:
df['original_language'].value_counts()

In [ ]:
def recommend(movie_name):
    pass

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(stop_words='english')

In [ ]:
df['overview'].fillna(' ',inplace=True)

In [ ]:
overview_matrix=cv.fit_transform(df['overview']).toarray()

In [ ]:
overview_matrix

In [ ]:
def recommend(movie_name):
    # find the index of this movie
    index_pos=df[df['title']==movie_name].index[0]
    
    # calulate similarity
    recommended_movie_index=sorted(list(enumerate(similarity_score[index_pos])),reverse=True,key=lambda x:x[1])[1:6]
    
    # movie name from index
    
    for i in recommended_movie_index:
        print(df.iloc[i[0]].title)
    

In [ ]:
recommend('Harry Potter and the Prisoner of Azkaban')

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
similarity_score=cosine_similarity(overview_matrix)

In [ ]:
similarity_score

In [ ]:
sorted(list(enumerate(similarity_score[200])),reverse=True,key=lambda x:x[1])[1:11]

In [ ]:
df.iloc[15].title

In [ ]:
df.columns

In [ ]:
df[['title','cast','crew','genres','keywords']]['cast'][0]

In [ ]:
['Action','Adventure','Science Fiction']

In [ ]:
# genres
from ast import literal_eval

In [ ]:
def clean_genre(x):
    genre=[]
    for i in literal_eval(x):
        genre.append(i['name'])
    return genre

In [ ]:
df['genres']=df['genres'].apply(clean_genre)

In [ ]:
df[['title','cast','crew','genres','keywords']]

In [ ]:
def clean_keyword(x):
    keywords=[]
    for i in literal_eval(x):
        keywords.append(i['name'])
    return keywords

In [ ]:
df['keywords']=df['keywords'].apply(clean_keyword)

In [ ]:
df['keywords']

In [ ]:
def clean_crew(x):
    for i in literal_eval(x):
        if i['job']=='Director':
            return i['name']
        

In [ ]:
df['crew']=df['crew'].apply(clean_crew)

In [ ]:
df['crew']

In [ ]:
def clean_cast(x):
    actor=[]
    count=0
    for i in literal_eval(x):
        if count<4:
            actor.append(i['name'])
            count+=1
        else:
            return actor
    

In [ ]:
df['cast']=df['cast'].apply(clean_cast)

In [ ]:
df['crew'][0]

In [ ]:
def clean(x):
    
    y=[]
    
    if type(x)==list:
        
        for i in x:
            
            i=i.lower()
            i=i.replace(" ","")
            y.append(i)
            
        return y
    if type(x)==str:
        
        x=x.lower()
        x=x.replace(" ","")
        
        return x

In [ ]:
features=['cast','crew','genres','keywords']

for i in features:
    
    df[i]=df[i].apply(clean)
    

In [ ]:
df[['cast','crew','genres','keywords']]

In [ ]:
def merge_cols(x):
    y=""
    
    y = y + x['crew'] + " " + " ".join(x['cast']) + " " + " ".join(x['genres']) + " " + " ".join(x['keywords'])
    
    return y

In [ ]:
df.dropna(subset=['cast','crew'],inplace=True)

In [ ]:
df['all']=df.apply(merge_cols,axis=1)

In [ ]:
cv=CountVectorizer(stop_words='english')

In [ ]:
movie_matrix=cv.fit_transform(df['all']).toarray()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity_matrix=cosine_similarity(movie_matrix)

In [ ]:
def recommend(movie_name):
    # find the index of this movie
    index_pos=df[df['title']==movie_name].index[0]
    
    # calulate similarity
    recommended_movie_index=sorted(list(enumerate(similarity_matrix[index_pos])),reverse=True,key=lambda x:x[1])[1:11]
    
    # movie name from index
    
    for i in recommended_movie_index:
        print(df.iloc[i[0]].title)
    

In [ ]:
recommend('Titanic')

In [ ]:
df[['id','title','all']].to_csv('movie.csv')

In [ ]:
import pickle

In [ ]:
pickle.dump(similarity_matrix,open('similarity_matrix.pkl','wb'))